In [ ]:
# extra top ranked book id's, etc from url

In [5]:
import sqlite3


con = sqlite3.connect("jmdict.db")

cur = con.cursor()

for row in cur.execute('SELECT furigana FROM jmdict where word = "儀式";'):
    print(row)

con.close()

('0:ぎ;1:しき',)


In [1]:
import jaconv

# Hiragana to Katakana
jaconv.hira2kata('ともえまみ')
# => 'トモエマミ'

# Hiragana to half-width Katakana
jaconv.hira2hkata('ともえまみ')
# => 'ﾄﾓｴﾏﾐ'

# Katakana to Hiragana
jaconv.kata2hira('巴マミ')

'巴まみ'

In [ ]:
from sudachipy import tokenizer
from sudachipy import dictionary
import sqlite3
import jaconv
import re
import requests
from requests.exceptions import HTTPError

In [12]:


def fetchRankedBooks():
    try:
        response = requests.get('https://pubserver2.herokuapp.com/api/v0.1/ranking/txt/2020/01')
        response.raise_for_status()
        # access JSOn content
        jsonResponse = response.json()
        print("Entire JSON response")
        #print(jsonResponse)

    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')
    except Exception as err:
        print(f'Other error occurred: {err}')

In [32]:
def fetchURL(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        # access JSOn content
        #jsonResponse = response.json()
        #print("Entire JSON response")
        #print(response)
        return response

    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')
    except Exception as err:
        print(f'Other error occurred: {err}')

In [14]:
%time fetchRankedBooks()

Entire JSON response
Wall time: 4.8 s


In [13]:
for row in jsonResponse:
    authors = ", ".join(row["authors"])
    print("book id " + str(row["book_id"]) + " title " + row["title"] + " authors " + authors)


book id 773 title こころ authors 夏目 漱石
book id 301 title 人間失格 authors 太宰 治
book id 127 title 羅生門 authors 芥川 竜之介
book id 456 title 銀河鉄道の夜 authors 宮沢 賢治
book id 789 title 吾輩は猫である authors 夏目 漱石
book id 2093 title ドグラ・マグラ authors 夢野 久作
book id 57624 title さぶ authors 山本 周五郎
book id 752 title 坊っちゃん authors 夏目 漱石
book id 1465 title 蟹工船 authors 小林 多喜二
book id 92 title 蜘蛛の糸 authors 芥川 竜之介
book id 1565 title 斜陽 authors 太宰 治
book id 57228 title 怪人二十面相 authors 江戸川 乱歩
book id 57817 title 日本婦道記 authors 山本 周五郎
book id 1567 title 走れメロス authors 太宰 治
book id 424 title 檸檬 authors 梶井 基次郎
book id 794 title 三四郎 authors 夏目 漱石
book id 799 title 夢十夜 authors 夏目 漱石
book id 45630 title 〔雨ニモマケズ〕 authors 宮沢 賢治
book id 2078 title 舞姫 authors 森 鴎外
book id 49866 title 変身 authors カフカ フランツ
book id 56648 title 人間椅子 authors 江戸川 乱歩
book id 43737 title 銀河鉄道の夜 authors 宮沢 賢治
book id 776 title 草枕 authors 夏目 漱石
book id 5016 title 源氏物語 authors 紫式部 
book id 52409 title 三国志 authors 吉川 英治
book id 56650 title Ｄ坂の殺人事件 authors 江戸川 乱歩
book

In [16]:
bookList = [row["book_id"] for row in jsonResponse]
print(bookList)

[773, 301, 127, 456, 789, 2093, 57624, 752, 1465, 92, 1565, 57228, 57817, 1567, 424, 794, 799, 45630, 2078, 49866, 56648, 43737, 776, 5016, 52409, 56650, 624, 59300, 43754, 2253, 52410, 42, 275, 47061, 58093, 935, 42286, 52504, 42620, 51215, 56866, 60202, 56388, 19, 50328, 69, 58277, 56669, 45245, 56645, 56646, 1927, 258, 42618, 462, 4803, 50909, 56651, 816, 470, 530, 60, 522, 55, 52411, 56143, 628, 46669, 56535, 1504, 52016, 59986, 975, 58813, 56698, 43014, 52412, 761, 56406, 56515, 52413, 59742, 1502, 51732, 473, 1869, 52976, 59636, 59819, 52414, 270, 605, 179, 59056, 56647, 52415, 59641, 52417, 52419, 882, 59429, 14, 52416, 236, 52418, 42934, 56559, 59427, 52420, 59364, 57405, 4673, 277, 637, 56389, 42621, 1058, 100, 623, 57649, 59055, 50131, 427, 15, 695, 785, 1317, 58102, 57612, 52973, 52421, 57181, 59057, 682, 49011, 307, 56641, 42773, 59384, 47148, 57105, 57291, 59058, 56041, 60082, 57190, 647, 16, 3804, 466, 240, 946, 57788, 56642, 57816, 1746, 2282, 894, 50049, 1669, 57347, 76

In [39]:
# test fetching a book
url = "https://pubserver2.herokuapp.com/api/v0.1/books/46115/content?format=txt"
bookData = fetchURL(url)
print(bookData.text)

青髯二百八十三人の妻
久生十蘭

-------------------------------------------------------
【テキスト中に現れる記号について】

《》：ルビ
（例）顎髯《あごひげ》

｜：ルビの付く文字列の始まりを特定する記号
（例）八百｜法《フラン》

［＃］：入力者注　主に外字の説明や、傍点の位置の指定
　　　（数字は、JIS X 0213の面区点番号またはUnicode、底本のページと行数）
（例）※［＃ローマ数字3、1-13-23］
-------------------------------------------------------

［＃５字下げ］一［＃「一」は中見出し］

　前大戦が終った翌年、まだ冬のままの二月のはじめ、パリの山手のレストランで働いているジャンヌ・ラコストという娘が、この十カ月以来、消息不明になっている姉のマダム・ビュイッソンの所在をたずねていた。スペインの国境に近いビアリッツにいる姉の一人息子が失明したという通知があったので、大急ぎで知らせなければならないと思ったのである。
　心あたりというほどのものはなかったが、前年の夏、休戦の二カ月ほど前、偶然、あるキャフェで姉と落ちあったとき、アンリ四世のような見事な顎髯《あごひげ》をはやした五十二三の紳士に紹介されたことがあったので、もしやと思って、そのほうをさがして見る気になった。その紳士はたしかアンドレ・シャルクロァといい、ヴェルサィユ市の南のガムベェという村に別荘があるというようなことを聞いた記憶がある。
　それで、とりあえずガムベェの村長に宛てて照会の手紙を出すと、折返して返事があった。そういう名の人物は居住していないが、手紙の趣にある風采《ふうさい》と齢《とし》恰好からおすと、三年前からトゥリック氏所有の別荘「エルミタージュ」を借りているラウール・デュポンのまちがいではないか。猶、ラウール・デュポンは去年の暮に来たきり、その後、一度もやって来ないと書いてあった。
　姉の消息を聞きだせるかと思っていたその当のひとまでが所在不明になっている。ジャンヌは考えにあまって、ガムベェの村長の手紙をもって警視庁の人事部へ姉の捜査をねがいに行った。
　前年の十月、馬鈴薯袋や防水紙の遮閉幕の蔭で息をひそめていた巴里が、やっとのことで四年という長い暗黒生活

In [38]:
for idx, line in enumerate(bookData.text.split("\n")):
    print(str(idx) +": " + line)

0: 青髯二百八十三人の妻
1: 久生十蘭
2: 
3: -------------------------------------------------------
4: 【テキスト中に現れる記号について】
5: 
6: 《》：ルビ
7: （例）顎髯《あごひげ》
8: 
9: ｜：ルビの付く文字列の始まりを特定する記号
10: （例）八百｜法《フラン》
11: 
12: ［＃］：入力者注　主に外字の説明や、傍点の位置の指定
13: 　　　（数字は、JIS X 0213の面区点番号またはUnicode、底本のページと行数）
14: （例）※［＃ローマ数字3、1-13-23］
15: -------------------------------------------------------
16: 
17: ［＃５字下げ］一［＃「一」は中見出し］
18: 
19: 　前大戦が終った翌年、まだ冬のままの二月のはじめ、パリの山手のレストランで働いているジャンヌ・ラコストという娘が、この十カ月以来、消息不明になっている姉のマダム・ビュイッソンの所在をたずねていた。スペインの国境に近いビアリッツにいる姉の一人息子が失明したという通知があったので、大急ぎで知らせなければならないと思ったのである。
20: 　心あたりというほどのものはなかったが、前年の夏、休戦の二カ月ほど前、偶然、あるキャフェで姉と落ちあったとき、アンリ四世のような見事な顎髯《あごひげ》をはやした五十二三の紳士に紹介されたことがあったので、もしやと思って、そのほうをさがして見る気になった。その紳士はたしかアンドレ・シャルクロァといい、ヴェルサィユ市の南のガムベェという村に別荘があるというようなことを聞いた記憶がある。
21: 　それで、とりあえずガムベェの村長に宛てて照会の手紙を出すと、折返して返事があった。そういう名の人物は居住していないが、手紙の趣にある風采《ふうさい》と齢《とし》恰好からおすと、三年前からトゥリック氏所有の別荘「エルミタージュ」を借りているラウール・デュポンのまちがいではないか。猶、ラウール・デュポンは去年の暮に来たきり、その後、一度もやって来ないと書いてあった。
22: 　姉の消息を聞きだせるかと思っていたその当のひとまでが所在不明になっている。ジャンヌは考えにあまって、ガムベェの村

# Sudachi Python Parser Example

In [210]:
from sudachipy import tokenizer
from sudachipy import dictionary

text = "考え二ヶ月べんきょう花火花粉たくさんきれいな寿司が食べたい。勉強をします。音楽がすきですから、よく CD を聞きます。ワンさん"

tokenizer_obj = dictionary.Dictionary().create()
mode = tokenizer.Tokenizer.SplitMode.C # mode.C is the one that splits things into the smallest form
#[m.surface() for m in tokenizer_obj.tokenize(text, mode)]
tokens = tokenizer_obj.tokenize(text, mode)
for token in tokens:
    print("surface {}, dict {}, reading {}, pos {}"\
          .format(token.surface(), token.dictionary_form(), token.reading_form(), token.part_of_speech()))

surface 考え, dict 考え, reading カンガエ, pos ('名詞', '普通名詞', '一般', '*', '*', '*')
surface 二, dict 二, reading ニ, pos ('名詞', '数詞', '*', '*', '*', '*')
surface ヶ月, dict ヶ月, reading カゲツ, pos ('名詞', '普通名詞', '助数詞可能', '*', '*', '*')
surface べんきょう, dict べんきょう, reading ベンキョウ, pos ('名詞', '普通名詞', 'サ変可能', '*', '*', '*')
surface 花火, dict 花火, reading ハナビ, pos ('名詞', '普通名詞', '一般', '*', '*', '*')
surface 花粉, dict 花粉, reading カフン, pos ('名詞', '普通名詞', '一般', '*', '*', '*')
surface たくさん, dict たくさん, reading タクサン, pos ('副詞', '*', '*', '*', '*', '*')
surface きれい, dict きれい, reading キレイ, pos ('形状詞', '一般', '*', '*', '*', '*')
surface な, dict だ, reading ナ, pos ('助動詞', '*', '*', '*', '助動詞-ダ', '連体形-一般')
surface 寿司, dict 寿司, reading スシ, pos ('名詞', '普通名詞', '一般', '*', '*', '*')
surface が, dict が, reading ガ, pos ('助詞', '格助詞', '*', '*', '*', '*')
surface 食べ, dict 食べる, reading タベ, pos ('動詞', '一般', '*', '*', '下一段-バ行', '連用形-一般')
surface たい, dict たい, reading タイ, pos ('助動詞', '*', '*', '*', '助動詞-タイ', '終止形-一般')
surface 。, dict 。, rea

In [146]:
def CleanUPAozoraText(text):
    rubyRegex = """(《.*?》)"""
    text = re.sub(rubyRegex, '', text)
    dashes = "-------------------------------------------------------"
    dashExpression = "(" + dashes + ".*" + dashes + ")"
    text = re.sub(dashExpression, '', text, 0, re.DOTALL)
    indentRegex = """(［＃.*?］)"""
    text = re.sub(indentRegex, '', text,)
    return text

# Magic to determine Wago vs Kango

In [98]:
def CalcTrueLen(furigana):
    # Function to return the true length of the reading.
    # Certain reading combinations can have multi characters but only count as 1 sound.
    # This likely indicates Kango. This function accounts for those combinations.
    # matching vowels くう ぐう　そう　ぞう　つう　のう　もう　ろう　ほう
    # けい　げい　せい　ぜい　てい ねい　めい　れい　へい
    vowels = ["あ", "い", "う", "え", "お",]
    glides1 = ["ょ", "ゅ", "ゃ"]
    glides2 = ["ょう", "ゅう"]
    furiLen = len(furigana)
    if furiLen == 1:
        return 1
    elif furiLen == 2:
        lastChar = furigana[1]
        if "しき" == furigana:
            return 1
        elif lastChar == "つ" or lastChar == "く" or lastChar == "ん" or lastChar == "っ":
            return 1
        elif lastChar in vowels:
            return 1
        elif lastChar in glides1:
            return 1
        else:
            return 2
    elif furiLen == 3:
        lastTwo = furigana[1:]
        secondChar = furigana[1]
        lastChar = furigana[2]
        if lastTwo in glides2:
            return 1
        elif not (lastChar == "つ" or lastChar == "く" or lastChar == "ん" or lastChar == "っ") \
            and (secondChar == "つ" or secondChar == "く" or secondChar == "ん" or secondChar == "っ"):
            # Second char in the last but last char not. 
            # (should only count as 2 if one or the other char matches the pattern. Not both)
            return 2
        elif (lastChar == "つ" or lastChar == "く" or lastChar == "ん" or lastChar == "っ") \
            and not (secondChar == "つ" or secondChar == "く" or secondChar == "ん" or secondChar == "っ"):
            # Last char in the list but second char not. 
            # (should only count as 2 if one or the other char matches the pattern. Not both)
            return 2
        else:
            return 3
    elif furiLen == 4:
        secondChar = furigana[1]
        lastChar = furigana[3]
        if (lastChar == "つ" or lastChar == "く" or lastChar == "ん" or lastChar == "っ") \
            and (secondChar == "つ" or secondChar == "く" or secondChar == "ん" or secondChar == "っ"):
            return 2
        else:
            return furiLen
    else:
        #just in case
        return furiLen
    

# Method to extract stats from text

In [213]:
# All of the math below is meant to match with the math used by JReadability
def ExtractStats(tokens, includeauxVerbs = False):
    numVerbs = 0
    numAux = 0
    numAuxVerbs = 0
    numKango = 0
    numWago = 0
    numGairaigo = 0
    sentenceAvgLen = 0
    numPeriods = 0
    numSpaces = 0
    verb = "動詞"
    dependent = "非自立可能"
    aux1 = "助動詞" # counts as an auxVerb
    aux2 = "助詞" # counts as an auxVerb
    auxVerb = "格助詞" # counts as an auxVerb
    adverb = "副詞"
    conjuction = "接続詞"
    symbol = "補助記号"
    space = "空白"
    naAdj = "形状詞"
    iAdj = "形容詞"
    noun = "名詞"
    properNoun = "固有名詞"
    pronoun = "代名詞"
    counter = "助数詞可能"
    connectingWord = "連体詞"
    suruVerb = "サ変可能"
    suffix = "接尾辞"
    hiraganaUniLow = u'\u3040'
    katakanaUniLow = u'\u30A0' 
    katakanaUniHi = u'\u30FF'
    kangoExceptionList = ["きれい"]
    alphabetRegexp = re.compile(r'[a-zA-Z]')
    kangoWords = {}
    wagoWords = {}
    verbWords = {}
    gairaigoWords = {}
    totalVerbsList = []
    
    
    for token in tokens:
        print(token.surface())
        if token.surface() == "。":
            numPeriods += 1
            continue
        
        
        posList = token.part_of_speech()
        if aux2 in posList or auxVerb in posList:
            numAux +=1
            numWago += 1
            wagoWords[token.surface()] = 1
            #print("adding {} to numWago".format(token.surface()))
        elif aux1 in posList:
            numAuxVerbs += 1
            numWago += 1
            wagoWords[token.surface()] = 1
            #print("adding {} to numWago".format(token.surface()))
            if includeauxVerbs:
                # Not sure if auxVerbs should count in the number of total auxillary words.
                # If includeauxVerbs then add it to the total
                numAux +=1
        elif verb in posList:
            dictForm = token.dictionary_form()
            #JReadability doesn't consider する/いる in the calculation.
            if dependent not in posList:
                numVerbs += 1
                verbWords[dictForm] = 1
                totalVerbsList.append(dictForm)
            numWago += 1
            wagoWords[token.surface()] = 1
            #print("adding {} to numWago".format(token.surface()))
        elif naAdj in posList:
            surface = token.surface()
            reading = jaconv.kata2hira(token.reading_form())
            if reading in kangoExceptionList:
                numKango += 1
                kangoWords[surface] = 1
                #print("adding {} to numKango".format(token.surface()))
            elif reading == surface:
                # Hiragana only word, so Wago
                numWago += 1
                wagoWords[surface] = 1
                #print("adding {} to numWago".format(surface))
            elif hiraganaUniLow <= min(surface) <= katakanaUniHi:
                # considering hiragana/katakana word as Wago.
                numWago += 1
                wagoWords[surface] = 1
                #print("adding {} to numWago".format(surface))
            else:
                # most likely Kango??
                numKango += 1
                kangoWords[surface] = 1
                #print("adding {} to numKango".format(token.surface()))
        elif adverb in posList:
            surface = token.surface()
            reading = jaconv.kata2hira(token.reading_form())
            #print(f'surface {surface}, reading {reading}, equal? {reading == surface}')
            if reading == surface:
                # Hiragana only word, so Wago
                numWago += 1
                wagoWords[surface] = 1
                #print("adding {} to numWago".format(surface))
            elif hiraganaUniLow <= min(surface) <= katakanaUniHi:
                # considering hiragana/katakana word as Wago.
                numWago += 1
                wagoWords[surface] = 1
                #print("adding {} to numWago".format(surface))
            else:
                # most likely Kango??
                numKango += 1
                kangoWords[surface] = 1
                #print("adding {} to numKango".format(token.surface()))
        elif conjuction in posList:
            numWago += 1
            wagoWords[token.surface()] = 1
            #print("adding {} to numWago".format(token.surface()))
        elif iAdj in posList:
            numWago += 1
            wagoWords[token.surface()] = 1
            #print("adding {} to numWago".format(token.surface()))
        elif properNoun in posList:
            # JReadability doesn't count these as wago or kango.
            continue
        elif suffix in posList:
            numWago += 1
            wagoWords[token.surface()] = 1
            #print("adding {} to numWago".format(token.surface()))
        elif pronoun in posList:
            numWago += 1
            wagoWords[token.surface()] = 1
            #print("adding {} to numWago".format(token.surface()))
        elif counter in posList:
            numWago += 1
            wagoWords[token.surface()] = 1
            #print("adding {} to numWago".format(token.surface()))
        elif connectingWord in posList:
            continue
            #numWago += 1
            #wagoWords[token.surface()] = 1
            #print("adding {} to numWago".format(token.surface()))
        elif noun in posList:
            #complicated case
            surface = token.surface()
            #print("found noun. posList {}".format(posList))
            if alphabetRegexp.search(surface):
                continue
            elif suruVerb in posList:
                # this checks for nouns that can take the suffix する to become verbs. Those nouns are usually Kango.
                numKango += 1
                kangoWords[surface] = 1
                #print("adding {} to numKango".format(surface))
            elif katakanaUniLow <= min(surface) <= katakanaUniHi:
                # considering katakana nouns as gairaigo
                numGairaigo += 1
                gairaigoWords[surface] = 1
            elif hiraganaUniLow <= min(surface) <= katakanaUniHi and not (hiraganaUniLow <= max(surface) <= katakanaUniHi):
                #word contains hiragana/katakana + kanji. Most likely wago
                numWago += 1
                wagoWords[token.surface()] = 1
            else:
                # Need to look at the reading compared to the surface to determine the character length
                # This is a hueristic so it's certainly not too accurate
                reading = jaconv.kata2hira(token.reading_form())
                #print(f'surface {surface}, reading {reading}, equal? {reading == surface}')
                if reading == surface:
                    # probably a hiragana only word, so wago
                    numWago += 1
                    wagoWords[surface] = 1
                    #print("adding {} to numWago".format(surface))
                    continue
                
                query = 'SELECT furigana FROM jmdict where word = "{}" and reading = "{}";'\
                                  .format(surface, reading)
                #print(query)
                row = cur.execute(query).fetchone()
                if row != None:
                    # Need deal with special cases ん、つ、く、しき
                    try:
                        print(f"found query result in db {row[0]}")
                        readingLen = 0
                        furiHelper = row[0].split(";") #e.g. '0:ぎ;1:しき'
                        for item in furiHelper:
                            ruby = item.split(":")
                            if "-" not in ruby[0]:
                                #normal case
                                furigana = ruby[1]
                                readingLen += CalcTrueLen(furigana)
                            else:
                                #special case like with 三和土|たたき|0-2:たたき
                                numRange = ruby[0].split("-")
                                # Is there a need to do something special with the numRange??
                                # Doing nothing for now.
                                furigana = ruby[1]
                                readingLen += CalcTrueLen(furigana)
                        
                        #again this is a hueristic and won't always be true
                        print(f'surface len {len(surface)}, reading len {readingLen}')
                        if len(surface) == readingLen:
                            numKango += 1
                            kangoWords[surface] = 1
                            print("adding {} to numKango".format(surface))
                        else:
                            numWago += 1
                            wagoWords[surface] = 1
                            print("adding {} to numWago".format(surface))
                                  
                    except Exception as err:
                        print(f'Error occurred: {err}')
                else:
                    print(f"couldn't find {surface} {reading}")
                    # word not found in jmdictfurigana so just use the reading
                    if len(surface) == len(reading):
                        numKango += 1
                        kangoWords[surface] = 1
                        print("adding {} to numKango".format(surface))
                    else:
                        numWago += 1
                        wagoWords[surface] = 1
                        print("adding {} to numWago".format(surface))
             
        elif space in posList:
            numSpaces += 1
        else:
            # It's probably a symbol. Do nothing
            print("What dis? {}".format(posList))
            continue
            
    # subtracting the numPeriods from the number of tokens before the divide would make this more accurate
    # but that's not what JReadability does, so leaving it this way for now.
    numTokens = len(tokens) - numSpaces
    sentenceAvgLen = numTokens/numPeriods
    print(f"{len(kangoWords.keys())} unique kango words found {kangoWords.keys()}")
    print(f"{len(wagoWords.keys())} unique wago words found {wagoWords.keys()}")
    print(f"{len(gairaigoWords.keys())} unique gairaigo words found {gairaigoWords.keys()}")
    print(f"{len(verbWords.keys())} unique verb words found {verbWords.keys()}")
    print(f"{len(totalVerbsList)} total verb words found {totalVerbsList}")
    print("result avgLen {}, kango {}, wago {}, gairaigo {} verbs {}, auxiliary {}, auxVerbs {}"\
          .format(sentenceAvgLen, numKango, numWago, numGairaigo, numVerbs, numAux, numAuxVerbs))
    return (sentenceAvgLen, numKango, numWago, numVerbs, numAux, numTokens)

# Calc Difficulty

In [214]:
def CalcDifficulty(sentenceAvgLen, numKango, numWago, numVerbs, numAux, numTokens):
    # Linear Regression Formula from http://jhlee.sakura.ne.jp/papers/lee-et-al2016rb.pdf
    # X = { 平 均 文 長 *-0.056}+{ 漢 語 率 *-0.126}+{ 和 語 率*-0.042}+{動詞率*-0.145}+{助詞率*-0.044}+11.724
    percentKango = numKango / numTokens * 100
    percentWago = numWago / numTokens * 100
    percentVerbs = numVerbs / numTokens * 100
    percentAuxVerbs = numAux / numTokens * 100
    difficulty = sentenceAvgLen*-0.056 + percentKango*-0.126 + percentWago*-0.042 \
        + percentVerbs*-0.145 + percentAuxVerbs*-0.044 + 11.724
    
    return difficulty

# Grade Aozora Content

In [215]:
# come back and change this cell to grab the actual text and feed that into the calculation
# connect to jmdictfurigana
con = sqlite3.connect("jmdict.db")
cur = con.cursor()
# call to Aozora goes here...

#text = "音楽がすきですから、よく CD を聞きます。日本がすきですから、日本語を勉強します。安かったですから、買いました。ディズニーランドは楽しかったです。教室は静かでした。わたしはラーメンがすきです。わたしはたばこがきらいです。ワンさんは日本語が上手です。わたしは料理が下手です。"
#text = "寿司が食べたい。"
#text = "わたしは料理が下手です。"
#text = "スペインの国境に近いビアリッツにいる姉の一人息子が失明したという通知があったので、大急ぎで知らせなければならないと思ったのである。"
text = aozoraText
tokens = tokenizer_obj.tokenize(text, mode)
(sentenceAvgLen, numKango, numWago, numVerbs, numAux, numTokens) = ExtractStats(tokens)

difficulty = CalcDifficulty(sentenceAvgLen, numKango, numWago, numVerbs, numAux, numTokens)
print("difficulty = {}".format(difficulty))

#Store the difficulty and other stats in sqlite

#finally close the connection to jmdictfurigana
con.close()

前大
戦
が
終っ
た
翌年
found query result in db 0:よく;1:ねん
surface len 2, reading len 2
adding 翌年 to numKango
、
What dis? ('補助記号', '読点', '*', '*', '*', '*')
まだ
冬
found query result in db 0:ふゆ
surface len 1, reading len 2
adding 冬 to numWago
の
まま
の
二
found query result in db 0:に
surface len 1, reading len 1
adding 二 to numKango
月
の
はじめ
、
What dis? ('補助記号', '読点', '*', '*', '*', '*')
パリ
の
山手
found query result in db 0:やま;1:て
surface len 2, reading len 3
adding 山手 to numWago
の
レストラン
で
働い
て
いる
ジャンヌ
・
What dis? ('補助記号', '一般', '*', '*', '*', '*')
ラコスト
と
いう
娘
found query result in db 0:むすめ
surface len 1, reading len 3
adding 娘 to numWago
が
、
What dis? ('補助記号', '読点', '*', '*', '*', '*')
この
十
found query result in db 0:じゅう
surface len 1, reading len 1
adding 十 to numKango
カ月
以来
found query result in db 0:い;1:らい
surface len 2, reading len 2
adding 以来 to numKango
、
What dis? ('補助記号', '読点', '*', '*', '*', '*')
消息不明
found query result in db 0:しょう;1:そく;2:ふ;3:めい
surface len 4, reading len 4
adding 消息不明 to numK

In [189]:
katakanaUniLow = u'\u30A0' 
katakanaUniHi = u'\u30FF'
word = "ヶ"
if katakanaUniLow <= min(word) <= katakanaUniHi:
    print("katakana")
else:
    print("not katakana")

katakana


In [153]:
# retry with modifications?? # delete this later
CalcDifficulty(sentenceAvgLen, numKango, numWago, numVerbs, numAux, numTokens)

1.6179066974595848

In [54]:
# unicode for kanji = [\u3400-\u4dbf\u4e00-\u9fff\uf900-\ufaff]

In [183]:
# remove extra stuff
aozoraText = """前大戦が終った翌年、まだ冬のままの二月のはじめ、パリの山手のレストランで働いているジャンヌ・ラコストという娘が、この十カ月以来、消息不明になっている姉のマダム・ビュイッソンの所在をたずねていた。スペインの国境に近いビアリッツにいる姉の一人息子が失明したという通知があったので、大急ぎで知らせなければならないと思ったのである。
　心あたりというほどのものはなかったが、前年の夏、休戦の二カ月ほど前、偶然、あるキャフェで姉と落ちあったとき、アンリ四世のような見事な顎髯《あごひげ》をはやした五十二三の紳士に紹介されたことがあったので、もしやと思って、そのほうをさがして見る気になった。その紳士はたしかアンドレ・シャルクロァといい、ヴェルサィユ市の南のガムベェという村に別荘があるというようなことを聞いた記憶がある。
　それで、とりあえずガムベェの村長に宛てて照会の手紙を出すと、折返して返事があった。そういう名の人物は居住していないが、手紙の趣にある風采《ふうさい》と齢《とし》恰好からおすと、三年前からトゥリック氏所有の別荘「エルミタージュ」を借りているラウール・デュポンのまちがいではないか。猶、ラウール・デュポンは去年の暮に来たきり、その後、一度もやって来ないと書いてあった。
　姉の消息を聞きだせるかと思っていたその当のひとまでが所在不明になっている。ジャンヌは考えにあまって、ガムベェの村長の手紙をもって警視庁の人事部へ姉の捜査をねがいに行った。
　前年の十月、馬鈴薯袋や防水紙の遮閉幕の蔭で息をひそめていた巴里が、やっとのことで四年という長い暗黒生活から解放されたが、治安状態はまだ闇のままであった。"""
aozoraText = CleanUPAozoraText(aozoraText)